<a href="https://colab.research.google.com/github/henryamster/KET-Legislative-Session-Archival-Footage-Scraper/blob/main/KET_Legislative_Session_Archival_Footage_Scraper_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download KET Legislative Session Archival Footage 2022

In [ ]:
#@title Grab video content
start = 1 #@param {type:"number"}
end = 200 #@param {type:"number"}
chunked_raw_url = 'https://5878fd1ed5422.streamlock.net/wordpress/_definst_/mp4:wgaos/wgaos_223291.mp4/media_w816149639_38.ts' #@param {type:"string"}

!pip install pysrt
import urllib.request
import os
import time
from google.colab import files
import pysrt
import re

stream_dir = '/content/stream'
if not os.path.exists(stream_dir):
    os.makedirs(stream_dir)
else:
    for f in os.listdir(stream_dir):
        os.remove(os.path.join(stream_dir, f))

url_trimmed= chunked_raw_url[0:chunked_raw_url.rfind('_')]

for i in range(start, end):
    time.sleep(1)
    url = f'{url_trimmed}_{i}.ts'
    urllib.request.urlretrieve(url, f'{stream_dir}/{i:04}.ts')
    


In [ ]:
#@title Merge all chunks into one, convert to MP4
!cat /content/stream/*.ts > all.ts
!ffmpeg -i all.ts -acodec copy -vcodec copy /content/stream/final.mp4
!cat all.ts | ffmpeg -f lavfi -i 'movie=pipe\\:0[out+subcc]' -map s /content/stream/subtitles.srt



In [ ]:
#@title Download video with closed captioning
files.download(f'{stream_dir}/final.mp4') 

In [55]:
#@title Convert captions to searchable text & download
subs = pysrt.open(f'{stream_dir}/subtitles.srt')
with open(f'{stream_dir}/text.html', 'w') as f:
    for sub in subs:
        f.write(re.sub('/{\\an7}/', ":",str(sub.text)))
        f.write('<br>')
files.download(f'{stream_dir}/text.html')
files.download(f'{stream_dir}/subtitles.srt')